In [ ]:
!pip uninstall -y kaggle
!pip install kaggle==1.5.6
!kaggle competitions download -c fcis-sc-deeplearning-competition
!unzip /content/fcis-sc-deeplearning-competition.zip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!unzip  '/content/drive/My Drive/test.zip'

In [ ]:
epochs = 10


from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard
import os.path
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input, decode_predictions
import numpy as np
import pandas as pd



# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# dimensions of our images.
#Inception input size
img_width, img_height = 299, 299

top_layers_checkpoint_path = 'cp.top.best.hdf5'
fine_tuned_checkpoint_path = 'cp.fine_tuned.best.hdf5'
new_extended_inception_weights = 'final_weights.hdf5'

train_data_dir = '/content/train/train'
TEST_DIR = '/content/val/val'


nb_train_samples = 3138
nb_validation_samples = 800

top_epochs = 100
fit_epochs = 100

batch_size = 24

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- we have 10 classes
predictions = Dense(10, activation='softmax')(x)

# this is the model we will train
model = Model(input=base_model.input, output=predictions)

if os.path.exists(top_layers_checkpoint_path):
	model.load_weights(top_layers_checkpoint_path)
	print ("Checkpoint '" + top_layers_checkpoint_path + "' loaded.")

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'], )

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')


#Save the model after every epoch.
mc_top = ModelCheckpoint(top_layers_checkpoint_path, monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)

#Save the TensorBoard logs.
tb = TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=True)

# train the model on the new data for a few epochs
#model.fit_generator(...)



if (os.path.exists('model1.tfl.meta')):
    model.load('./model1.tfl')
else:
    model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples // batch_size,
    nb_epoch=top_epochs,
    validation_data=0,
    nb_val_samples=nb_validation_samples // batch_size,
    callbacks=[mc_top, tb])
    model.save('mode1.tfl')


# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)


#Save the model after every epoch.
mc_fit = ModelCheckpoint(fine_tuned_checkpoint_path, monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)


if os.path.exists(fine_tuned_checkpoint_path):
	model.load_weights(fine_tuned_checkpoint_path)
	print ("Checkpoint '" + fine_tuned_checkpoint_path + "' loaded.")

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 172 layers and unfreeze the rest:
for layer in model.layers[:172]:
   layer.trainable = False
for layer in model.layers[172:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
#model.fit_generator(...)

if (os.path.exists('model2.tfl.meta')):
    model.load('./model2.tfl')
else:
    model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples // batch_size,
    nb_epoch=fit_epochs,
    validation_data=0,
    nb_val_samples=nb_validation_samples // batch_size,
    callbacks=[mc_fit, tb])    
    model.save('mode2.tfl')



def preprocess_input_inc(x):
    x /= 255.
    x -= 0.5
    x *= 2.
    return x

testing_pre = []

for imagep in os.listdir(TEST_DIR):
    image_path = os.path.join(TEST_DIR, imagep)
    img = image.load_img(image_path, target_size=(299, 299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    x = preprocess_input_inc(x)
    preds = model.predict(x)
    print(imagep, preds, np.argmax(preds) + 1)
    testing_pre.append([imagep,np.argmax(preds) + 1])

Csv = pd.DataFrame(testing_pre)
Csv.to_csv("subm1.csv")